ID3 Desicion Tree

*   Training file - 'id3-train.dat'
*   Testing file - 'id3-test.dat'

In these files, only non-space characters are relevant. The first line contains the attribute names. All the other lines are example instances to be used for the algorithm. Each column holds values of the attributes, whereas the last column holds the class label for that instance.

In a decision tree, if you reach a leaf node but still have examples that belong to different classes, then choose the most frequent class (among the instances at the leaf node). If you reach a leaf node in the decision tree and have no examples left or the examples are equally split among multiple classes, then choose the class that is most frequent in the entire training set. You do not need to implement pruning. Also, don’t forget to use logarithm base 2 when computing entropy and set (0 log 0) to 0.

Write the code in the following code block, structure is provided. Instructions on the steps to follow are provided as comments. The code should output the following 3 things:

*   Print the Decision Tree created, in the following example format:

    ```
    attr1 = 0 :
        attr2 = 0 :
            attr3 = 0 : 1 -- 4
            attr3 = 1 : 0 -- 9
        attr2 = 1 :
            attr4 = 0 : 0 -- 2
            attr4 = 1 : 1 -- 10
    attr1 = 1 :
        attr2 = 1 : 1 -- 17

    ```

*   Accuracy on the Training data = x %
*   Accuracy on the Test data = x %


Import libraries

In [ ]:
# import all required libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import array

In [ ]:
# Assume that the data files are in the following folder -- 
basePath = "/Users/martgom/Documents/UTSA/FALL-2023/Artificial Intelligence/HW3"


In [ ]:
# Data file name variables
train = basePath + "/id3-train.dat"
test = basePath + "/id3-test.dat"


In [ ]:
#NODE
class Node:
    def __init__(self, attribute = None, value = None, leaf_class = None):
        self.attribute = attribute
        self.value = value
        self.leaf_class = leaf_class
        self.children = {}
        
#calculate the disorder of dataset
def calculate_e(data):
    total_ins = len(data)
    class_lbl = [instance[-1] for instance in data]
    class_c = {}
    
    for lbl in class_lbl:
        if lbl in class_c:
            class_c[lbl]+=1
        else:
            class_c[lbl] = 1
    disorder = 0
    for count in class_c.values():
        prob = count / total_ins
        disorder = disorder - prob * math.log2(prob)
    return disorder

#calculate information 
def calculate_info_gain(data, att_index):
    total_e = calculate_e(data)
    att_values = [0,1]
    weight_e = 0
    
    for value in att_values:
        sub = [instance for instance in data if instance[att_index] == value]
        sub_e = calculate_e(sub)
        sub_weight = len(sub)/len(data)
        weight_e += sub_weight *sub_e
    return total_e - weight_e

# Pseudocode for the ID3 algorithm. Use this to create function(s).
def ID3(data, root, attributesRemaining):
    # If you reach a leaf node in the decision tree and have no examples left or the examples are equally split among multiple classes
    if not data:
        # Choose and the class that is most frequent in the entire training set and return the updated tree
        return root
    # If all the instances have only one class label
    class_lbl = [instance[-1] for instance in data]
    if len(set(class_lbl)) == 1:
        # Make this as the leaf node and use the label as the class value of the node and return the updated tree
        root.leaf_class = class_lbl[0]
        return root
    # If you reached a leaf node but still have examples that belong to different classes (there are no remaining attributes to be split)
    if not attributesRemaining:
        # Assign the most frequent class among the instances at the leaf node and return the updated tree
        root.leaf_class = max(set(class_lbl), key=class_lbl.count)
        return root
    # Find the best attribute to split by calculating the maximum information gain from the attributes remaining by calculating the entropy
    best_at_index , best_info_gain = -1, -1
    
    for attribute_index in attributesRemaining:
        info_gain = calculate_info_gain(data, attribute_index)
        if info_gain > best_info_gain:
            best_at_index = attribute_index
            best_info_gain = info_gain
        
        if best_at_index == -1:
            root.leaf_class = max(set(class_lbl), key = class_lbl.count)
            return root
        best_at = best_at_index
        root.attribute = best_at
    # Split the tree using the best attribute and recursively call the ID3 function using DFS to fill the sub-tree
    for attribute_value in [0 , 1]:
        sub = [instance for instance in data if instance[best_at]==attribute_value]
        if not sub:
            leaf_class = max(set(class_lbl), key = class_lbl.count)
            root.children[attribute_value] = Node(leaf_class = leaf_class)
        else:
            root.children[attribute_value] = ID3(sub, Node(), [attribute for attribute in attributesRemaining if attribute !=best_at])
            
    # return the root as the tree
    return root

    #print desicion tree
def print_decision(node, level = 0):
    if node.leaf_class is not None:
        print(f'{node.leaf_class} --- {len(content)}')
    else:
        for value, child_nd in node.children.items():
            print('\t' * level + f'attribute-{node.attribute} = {value}:', end=' ')
            print_decision(child_nd, level +1)

#read file
def read_data_file(filename) :
    with open(filename, 'r') as file:
        #skip first line
        next(file)
        content = [list(map(int, line.strip().split())) for line in file.readlines()]
    return content
#read content callinf read_data_file method
content = read_data_file(train)
#get number of attributes
num_att = len(content[0])-1
#list of attributes
train_att = list(range(num_att))

#create the desicion tree
root = Node()
dec_tree = ID3(content, root, train_att)

#print
print_decision(dec_tree)




        
    


In [ ]:
# Following is the base code structure. Feel free to change the code structure as you see fit, maybe even create more functions.
#predict labels of the tree
def predict_lbl(tree, attributes):
    if tree.leaf_class is not None:
        return tree.leaf_class
    
    attribute_value = attributes[tree.attribute]
    
    if attribute_value in tree.children:
        return predict_lbl(tree.children[attribute_value], attributes)
    else:
        
        return tree.leaf_class
    
        
# Read the first line in the training data file, to get the number of attributes

# Read all the training instances and the ground truth class labels.
# Create the decision tree by implementing the ID3 algorithm. Pseudocode provided above.
def print_desicion_tree(node, att_names, level = 0):
    if node.leaf_class is not None:
        return
    
    for value, child_nd in node.children.items():
        prefix = " " * level
        att_nm = att_names[node.attribute]
        data_len = len(child_nd.data) if hasattr(child_nd, "data") else 0
        print(f"{prefix}{att_nm} = {value} : {data_len} -- {child_nd.leaf_class}")
        print_desicion_tree(child_nd, att_names, level+1)
    
#calculate accuracy
def cal_accuracy( content, tree):
    
    predictions = 0
    total = len(content)
    # For each training instance, predict the output label
    for instance in content:
        #do not include last column and get label from the current database
        att= instance[:-1]
        current_class = instance[-1]
        pred_class = predict_lbl(tree, att)
        
        # Compare it with the ground truth class label and calculate the accuracy accordingly
        #check using the tree
        if pred_class == current_class:
            
            predictions +=1
        #calculate and return accuracy
        #print(predictions)
        #print(total)
    accuracy = ((predictions / total) * 100)
    return accuracy
    
            
        
#main method 
def main():
    content = read_data_file(train)
    #print(content)
    num_att = len(content[0] )-1
    train_att = list(range(num_att))
    
    #create the tree
    root = Node()
    dt = ID3(content,root, train_att)
    #create attributes defined
    attributes = []

    for i in range(1, 7):
        attribute = f"attribute-{i}"
        attributes.append(attribute)
    attributes.append("Class")
    #print tree
    print_desicion_tree(dt, attributes)
    
    train_acc = cal_accuracy(content, dt)
    if train_acc is not None:
        print(f"Accuracy on Train: = {train_acc:.2f}%")
    else:
        print("ERROR Train.")
    
    data_test = read_data_file(test)
    #print accuracy of the training data
    test_acc = cal_accuracy(data_test, dt)
    if test_acc is not None:
        print(f"Accuracy on Test: = {test_acc:.2f}%")
    else:
        print("ERROR.")
if __name__ == "__main__":
    main()





Learning Curve representation graph

In [ ]:
# Loop through to select the number of instances 'x' in increments of 40
def generate_curve(train_data, test_data, step = 40):
    n_instances = len(train_data)
    n_att = len(train_data[0])-1
    train_att = list(range(n_att))
    
    xv =[]
    yv = []
    # For each 'x',
    for x in range(step, n_instances +1 ,step):
        # Randomly select 'x' instances
        random_data = random.sample(train_data, x)
    
        # Create the ID3 decision tree using those instances
        dt = ID3(random_data, root, train_att)
        # Calculate the accuracy of the ID3 tree created on the Test data
        accuracy = cal_accuracy(test_data, dt)
        
        xv.append(x)
        yv.append(accuracy)
    return xv,yv

test_data = read_data_file(test)
xv, yv = generate_curve(content, test_data)        
# Plot the learning curve using the accuracy values
plt.plot(xv, yv, marker = '.')
plt.title("Learning Curve Graph")

# X-axis will be the number of training instances used for creating the tree
plt.xlabel("Number of instances")
# Y-axis will be the accuracy in % on the Test data
plt.ylabel("Test-Acurracy Percentage")
plt.grid(True), plt.show()


